In [24]:
!pip install psycopg2-binary gspread gspread-dataframe pandas oauth2client

In [25]:
# Celda de Imports (debería ser tu primera o segunda celda de código)
import psycopg2
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
import json
import os
from google.colab import auth
from google.auth import default

In [26]:
import gspread
import json
import os

# Autenticación inteligente: usa la cuenta del robot en GitHub o tu cuenta en Colab
if 'GCP_SA_KEY' in os.environ:
    print("Autenticando con Cuenta de Servicio desde GitHub Secrets...")
    keyfile_dict = json.loads(os.environ['GCP_SA_KEY'])
    gc = gspread.service_account_from_dict(keyfile_dict)
else:
    print("Autenticando con usuario de Colab...")
    from google.colab import auth
    auth.authenticate_user()
    from google.auth import default
    creds, _ = default()
    gc = gspread.authorize(creds)

Autenticando con usuario de Colab...


In [27]:
import os

# Esta sección busca la contraseña.
# Si está en GitHub, la lee de los "secrets".
# Si está en Colab, la lee del gestor de secretos de Colab.
# Si no, la pones a mano (menos seguro).

if 'DB_PASSWORD' in os.environ:
    db_password = os.environ['DB_PASSWORD']
    print("Contraseña leída desde GitHub Secrets.")
else:
    try:
        from google.colab import userdata
        db_password = userdata.get('DB_PASSWORD')
        print("Contraseña leída desde Colab Secrets.")
    except:
        db_password = "TU_CONTRASEÑA_AQUÍ" # <-- Mantén esto para pruebas locales
        print("Usando contraseña local.")

Contraseña leída desde Colab Secrets.


In [28]:
# --------------- CONFIGURACIÓN (Usa la query simple para la prueba) ---------------
db_host = 'ec2-52-206-224-222.compute-1.amazonaws.com'
db_name = 'd77mgnf951v3g0'
db_user = 'reports'
sheet_name = 'Prueba automatización COLAB' # <-- Asegúrate que el nombre sea exacto
sql_query = """
    SELECT id, name, origin, phone
    FROM stores
    LIMIT 10
"""
# ----------------------------------------------------------------------------------

print("Iniciando proceso...")
conn = None # Inicializamos la variable de conexión

try:
    # 1. Conectar a PostgreSQL
    print("Checkpoint 1: Conectando a la base de datos...")
    conn = psycopg2.connect(
        host=db_host,
        dbname=db_name,
        user=db_user,
        password=db_password,
        port=5432
    )
    print("Checkpoint 2: Conexión a la base de datos exitosa.")

    # 2. Ejecutar la query y cargarla en un DataFrame
    print("Checkpoint 3: Ejecutando la consulta SQL...")
    df = pd.read_sql_query(sql_query, conn)
    print(f"Checkpoint 4: Consulta ejecutada. Se encontraron {len(df)} filas.")

    # 3. Conectar a Google Sheets
    print(f"Checkpoint 5: Abriendo Google Sheet: '{sheet_name}'...")
    spreadsheet = gc.open(sheet_name)
    print("Checkpoint 6: Hoja de cálculo abierta con éxito.")

    worksheet = spreadsheet.get_worksheet(0) # Obtiene la primera pestaña
    print(f"Checkpoint 7: Pestaña '{worksheet.title}' seleccionada.")

    print("Checkpoint 8: Limpiando la hoja y escribiendo los nuevos datos...")
    worksheet.clear()
    set_with_dataframe(worksheet, df)
    print("Checkpoint 9: La escritura de datos ha finalizado.")

    print("\n¡Proceso completado con éxito! Tu Google Sheet ha sido actualizado.")

except Exception as e:
    print(f"\nERROR: Ha ocurrido un problema crítico.")
    print(f"Detalle del error: {e}")
    raise e # <-- MUY IMPORTANTE: Esto hará que el job de GitHub falle y nos muestre el error.

finally:
    # 4. Asegurarse de que la conexión siempre se cierre
    if conn is not None:
        conn.close()
        print("Conexión a la base de datos cerrada.")

Iniciando proceso...
Checkpoint 1: Conectando a la base de datos...
Checkpoint 2: Conexión a la base de datos exitosa.
Checkpoint 3: Ejecutando la consulta SQL...
Checkpoint 4: Consulta ejecutada. Se encontraron 10 filas.
Checkpoint 5: Abriendo Google Sheet: 'Prueba automatización COLAB'...


/tmp/ipython-input-1423833759.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


Checkpoint 6: Hoja de cálculo abierta con éxito.
Checkpoint 7: Pestaña 'Hoja 1' seleccionada.
Checkpoint 8: Limpiando la hoja y escribiendo los nuevos datos...
Checkpoint 9: La escritura de datos ha finalizado.

¡Proceso completado con éxito! Tu Google Sheet ha sido actualizado.
Conexión a la base de datos cerrada.
